In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from IPython.display import Audio
from scipy.io.wavfile import write

In [3]:
import os

In [4]:
class_to_tgt = {
    'zero': 0,
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9
}

In [5]:
mels_dir = 'audio_diff_gan/mels'

In [6]:
targets = []
filenames = []

In [7]:
from glob import glob

In [50]:
for dir in os.listdir(f'{mels_dir}'):
    class_filenames = glob(f'{mels_dir}/{dir}/**/*.spec.npy', recursive=True)
    targets += [class_to_tgt[dir]] * len(class_filenames)
    filenames += class_filenames
    print(f'in {dir}: target={class_to_tgt[dir]}, len={len(class_filenames)}')

in zero: target=0, len=2376
in five: target=5, len=0
in six: target=6, len=0
in nine: target=9, len=0
in four: target=4, len=0
in two: target=2, len=402
in seven: target=7, len=0
in three: target=3, len=0
in one: target=1, len=2370
in eight: target=8, len=0


In [2]:
from tqdm import tqdm

In [4]:
from glob import glob

In [5]:
filenames = glob(f'ddgan/audio_diff_gan/mels/**/*.spec.npy', recursive=True)

In [6]:
len(filenames)

23666

In [10]:
import torch
import numpy as np

In [12]:
for file in tqdm(filenames):
    spectrogram = torch.from_numpy(np.load(file))
    assert(spectrogram.shape == torch.Size([1, 128, 128]))

100%|██████████| 23666/23666 [00:19<00:00, 1197.88it/s]


In [56]:
idx = 19000
spec_filename = filenames[idx]
spectrogram = torch.from_numpy(np.load(spec_filename))
target = targets[idx]
spectrogram, target

(tensor([[[-11.5129, -11.5129, -11.5129,  ..., -11.5129, -11.5129, -11.5129],
          [-11.5129, -11.5129, -11.5129,  ..., -11.5129, -11.5129, -11.5129],
          [-11.5129, -11.5129, -11.5129,  ..., -11.5129, -11.5129, -11.5129],
          ...,
          [-11.5129, -11.5129, -11.5129,  ..., -11.5129, -11.5129, -11.5129],
          [-11.5129, -11.5129, -11.5129,  ..., -11.5129, -11.5129, -11.5129],
          [-11.5129, -11.5129, -11.5129,  ..., -11.5129, -11.5129, -11.5129]]]),
 1)

In [14]:
import os

In [15]:
exp_path = './'
with open(os.path.join(exp_path, 'loss_logs.txt'), 'a') as logs_file:
    logs_file.write('epoch {} iteration{}, G Loss: {}, D Loss: {}\n'.format(3, 1, 2, 4))

In [57]:
spectrogram.shape

torch.Size([1, 128, 128])

In [42]:
from audio_diff_gan.tools import plot_melspecs

In [60]:
plot_melspecs(spectrogram, dir='mel_samples', tags='one')

In [47]:
# Load checkpoint
hifigan = torch.hub.load("bshall/hifigan:main", "hifigan_hubert_soft").cuda()
# Load mel-spectrogram
# wav, sr = hifigan.generate(mel_padded)

Using cache found in /home/yaoaksyonov/.cache/torch/hub/bshall_hifigan_main


Removing weight norm...


In [61]:
wav, sr = hifigan.generate(spectrogram.cuda())

In [62]:
Audio(wav.squeeze((0, 1)).cpu(), rate=sr)

In [28]:
Audio(np.full((1, 16000), 0), rate=sr)

In [29]:
Audio(wav[..., 3040:19040].squeeze((0, 1)).cpu(), rate=sr)

In [31]:
import torch.nn.functional as F

In [36]:
audio = F.pad(wav[..., 3040:19040].squeeze((0, 1)).cpu(), (790, 790), "constant", value=0)

In [37]:
audio.shape

torch.Size([17580])

In [38]:
Audio(audio.cpu(), rate=sr)

In [20]:
write("sample.wav", sr, wav.squeeze((0, 1)).cpu())

AttributeError: 'torch.dtype' object has no attribute 'kind'